In [ ]:
K8S_PROXY_ADDR='127.0.0.1:8001'
K8S_NAMESPACE='mdt'
APP_NAME='hello-world'
MODEL_NAME='sum-model'
REPO_URL='658391232643.dkr.ecr.us-west-2.amazonaws.com'

Create a Clipper Cluster

In [ ]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
manager = KubernetesContainerManager(kubernetes_proxy_addr=K8S_PROXY_ADDR,namespace=K8S_NAMESPACE)
clipper_conn = ClipperConnection(manager)
clipper_conn.start_clipper()

Connect to an existing cluster

In [ ]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
manager = KubernetesContainerManager(kubernetes_proxy_addr=K8S_PROXY_ADDR,namespace=K8S_NAMESPACE)
clipper_conn = ClipperConnection(manager)
clipper_conn.connect()

Create an Application

In [ ]:
clipper_conn.register_application(name=APP_NAME, input_type="doubles", default_output="-1.0", slo_micros=100000)
print clipper_conn.get_all_apps()

As the K8s cluster is on AWS, using ECR as the container repo here, need login to ECR first

In [ ]:
!$(aws ecr get-login --no-include-email --region us-west-2)

Create a simple python predict function and deploy it, note, the repo named `default-cluster-<model-name>` need be created in advance

In [ ]:
from clipper_admin.deployers import python as python_deployer
from clipper_admin import ClipperConnection, KubernetesContainerManager

manager = KubernetesContainerManager(kubernetes_proxy_addr=K8S_PROXY_ADDR,namespace=K8S_NAMESPACE)
clipper_conn = ClipperConnection(manager)
clipper_conn.connect()
def feature_sum(xs):
    return [str(sum(x)) for x in xs]

python_deployer.deploy_python_closure(clipper_conn, 
                                      name=MODEL_NAME, 
                                      version=2, 
                                      input_type="doubles", 
                                      func=feature_sum,
                                     registry=REPO_URL)

Link the mode to the application will create an endpoint and then you can call the REST.  `http://<query_address>/<application-name>/predict`

In [ ]:
clipper_conn.link_model_to_app(app_name=APP_NAME, model_name=MODEL_NAME)

Call REST post to do the inference

In [ ]:
import requests, json, numpy as np
url = "http://{}/api/v1/namespaces/{}/services/query-frontend-at-default-cluster:1337/proxy/{}/predict".format(K8S_PROXY_ADDR,K8S_NAMESPACE,APP_NAME)
# or clipper_conn.get_query_addr()
headers = {"Content-type": "application/json"}
data = json.dumps({"input": list(np.random.random(10))})
print(data)
requests.post(url, headers=headers, data=data).json()


List all models and apps with detailed information here

In [ ]:
for app in clipper_conn.get_all_apps(verbose=True):
    print json.dumps(app, indent=2)
for model in clipper_conn.get_all_models(verbose=True):
    print json.dumps(model, indent=2)

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

print clipper_conn.inspect_instance()

np.random.seed(19680801)

# example data
mu = 100  # mean of distribution
sigma = 15  # standard deviation of distribution
x = mu + sigma * np.random.randn(437)

num_bins = 50

fig, ax = plt.subplots()

# the histogram of the data
n, bins, patches = ax.hist(x, num_bins)

# add a 'best fit' line
y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
     np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
ax.plot(bins, y, '--')
ax.set_xlabel('Smarts')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of IQ: $\mu=100$, $\sigma=15$')

# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
# plt.show()

Stop Models and Applications

In [ ]:
clipper_conn.stop_models(MODEL_NAME)

In [ ]:
clipper_conn.delete_application(APP_NAME)


stop the clipper cluster will delete all the deployment of clipper

In [ ]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
manager = KubernetesContainerManager(kubernetes_proxy_addr='127.0.0.1:8001',namespace='mdt')
clipper_conn = ClipperConnection(manager)
clipper_conn.stop_all()